In [ ]:
import scanpy as sc
import transcriptomic_clustering as tc 

In [ ]:
tasic_adata = sc.read_h5ad('./data/tasic2016counts_sparse.h5ad')

# Normalize
normalized_adata = tc.normalize(tasic_adata)
print(normalized_adata)


In [ ]:
tasic_adata = sc.read_h5ad('./data/tasic2016counts_sparse.h5ad')

# Normalize
normalized_adata = tc.normalize(tasic_adata)

# Highly Variant Genes
means, variances, gene_mask = tc.get_means_vars_genes(adata=normalized_adata)
tc.highly_variable_genes(adata=normalized_adata, 
                         means=means, variances=variances, 
                         gene_mask=gene_mask, max_genes=3000)

# PCA
(components, explained_variance_ratio, explained_variance, means) = \
    tc.pca(normalized_adata, n_comps=25, cell_select=1000, use_highly_variable=True, svd_solver='arpack')
print(components)

## Filter Known Modes

In [ ]:
known_modes = components[[24]] # select last component as a known mode as an example
components = tc.filter_known_modes(components, known_modes)
print(components.shape)

## Projection

In [ ]:
projected_adata = tc.project(normalized_adata, components, means)
print(projected_adata)

## Louvain Clustering

In [ ]:
cluster_by_obs, obs_by_cluster, graph, qc = tc.cluster_louvain(projected_adata, 30, n_jobs=8)

## Merging

In [ ]:
thresholds = {
    'q1_thresh': 0.5,
    'q2_thresh': None,
    'cluster_size_thresh': 6,
    'qdiff_thresh': 0.7,
    'padj_thresh': 0.05,
    'lfc_thresh': 1.0,
    'score_thresh': 40,
    'low_thresh': 1
}

cluster_assignments_after_merging = tc.merge_clusters(
    adata_norm=normalized_adata,
    adata_reduced=projected_adata,
    cluster_assignments=obs_by_cluster,
    cluster_by_obs=cluster_by_obs,
    thresholds=thresholds,
    de_method='ebayes'
)

## Hierarchical Sorting

In [ ]:
import numpy as np
cluster_by_obs_after_merging = np.zeros(len(cluster_by_obs), dtype=int)
for cluster, obs in cluster_assignments_after_merging.items():
    cluster_by_obs_after_merging[obs] = cluster
cluster_means, _, _ = tc.get_cluster_means(projected_adata, cluster_assignments_after_merging, cluster_by_obs_after_merging)
linkage, labels = tc.hclust(cluster_means)

In [ ]:
from scipy.cluster.hierarchy import dendrogram
from matplotlib import pyplot as plt
fig = plt.figure()
dendrogram(linkage, labels=labels)
plt.show()